In [5]:
from utils.preprocessing import preprocessingV1
from utils.modelization import saveModel,loadModel,submitModel

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn import preprocessing

In [41]:
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectPercentile

from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler



In [3]:
df_train = pd.read_csv("datasets/train_radiomics_hipocamp.csv")
df_test = pd.read_csv("datasets/test_radiomics_hipocamp.csv")

X_train, y_train = preprocessingV1(df_train)

X_test = preprocessingV1(df_test,False)

In [59]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=123)
model = LogisticRegression(random_state=123)

fs = SelectKBest(score_func=f_classif)
pipeline = Pipeline(steps=[('anova',fs),  ('scaler', StandardScaler()),  ('lr', model)])

grid = dict()


grid = {
        'scaler' : [StandardScaler(),MinMaxScaler(),RobustScaler()],

        'anova__score_func': [ f_classif],
        #'anova__score_func': [ f_classif,mutual_info_classif], # mutual_info_classif lenta e não tem melhores resultados
        'anova__k': [i for i in range(8, 12)], # Number of features to select
       # 'anova__k': [i for i in range(5, 20)], 
        #'anova__k': [15,100,500,1000,1500],  
        
       # 'lr__class_weight': ['balanced',None], # balanced nunca me deu melhor resultados
        'lr__max_iter' : [1000], # As vezes nao convergia com 100
     #   'lr__multi_class': ['multinomial'], # nao é preciso especificar isto, liblinear é o unico 'ovr' dos 3
        'lr__solver': ['lbfgs','newton-cg','liblinear'], # 
        #'lr__C':[0.01, 0.1,1,10,100] # smaller values means stronger regularizaztion, 
        'lr__C':[1,10,100,1000] # 1-100 had better results
    }

search = GridSearchCV(pipeline, grid, scoring='f1_macro', n_jobs=-1, cv=cv, refit=True)
results = search.fit(X_train, y_train)

#Best Mean F1: 0.354
#Best Config: {'anova__k': 8, 'anova__score_func': <function f_classif at 0x000001CD99A33560>, 'lr__C': 100, 'lr__max_iter': 1000, 'lr__solver': 'newton-cg', 'scaler': MinMaxScaler()}
print('Best Mean Macro F1: %.3f' % results.best_score_)
print('Best Config: %s' % results.best_params_)

Best Mean F1: 0.354
Best Config: {'anova__k': 8, 'anova__score_func': <function f_classif at 0x000001CD99A33560>, 'lr__C': 100, 'lr__max_iter': 1000, 'lr__solver': 'newton-cg', 'scaler': MinMaxScaler()}


c:\Users\gusta\.conda\envs\mypython3v\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [30]:

#Best Mean Accuracy: 0.474
#Best Config: {'anova__k': 13, 'anova__score_func': <function f_classif at 0x000002188200B740>, 'lr__C': 0.1, 'lr__solver': 'lbfgs'}

# so teve 0.21818



#Best Mean F1: 0.340
#Best Config: {'anova__k': 15, 'anova__score_func': <function f_classif at 0x000001CD99A33560>, 'lr__C': 0.1, 'lr__class_weight': None, 'lr__solver': 'newton-cg'}


#Best Mean F1: 0.344
#Best Config: {'anova__k': 18, 'anova__score_func': <function f_classif at 0x000001CD99A33560>, 'lr__C': 0.01, 'lr__multi_class': 'multinomial', 'lr__solver': 'lbfgs'}

#Best Mean F1: 0.343
#Best Config: {'anova__k': 21, 'anova__score_func': <function f_classif at 0x000001CD99A33560>, 'lr__C': 1, 'lr__max_iter': 1000, 'lr__multi_class': 'multinomial', 'lr__solver': 'lbfgs'}

#Best Mean F1: 0.349
#Best Config: {'anova__k': 10, 'anova__score_func': <function f_classif at 0x000001CD99A33560>, 'lr__C': 1, 'lr__max_iter': 1000, 'lr__multi_class': 'multinomial', 'lr__solver': 'lbfgs'}

#Best Mean F1: 0.353
#Best Config: {'anova__k': 10, 'anova__score_func': <function f_classif at 0x000001CD99A33560>, 'lr__C': 1, 'lr__max_iter': 1000, 'lr__multi_class': 'multinomial', 'lr__solver': 'lbfgs', 'scaler': MinMaxScaler()}

#Best Mean F1: 0.354
#Best Config: {'anova__k': 9, 'anova__score_func': <function f_classif at 0x000001CD99A33560>, 'lr__C': 100, 'lr__max_iter': 1000, 'lr__multi_class': 'multinomial', 'lr__solver': 'lbfgs', 'scaler': MinMaxScaler()}




# Submeti este por ter o C mais baixo, resultado 0.27707

# grid = {
#         'scaler' : [StandardScaler(),MinMaxScaler(),RobustScaler()],
#         'anova__score_func': [ f_classif],

#         'anova__k': [i for i in range(5, 15)],
        
#         'lr__max_iter' : [1000],
#         'lr__solver': ['lbfgs','newton-cg','liblinear'],
#         'lr__C':[0.01, 0.1]
#     }
#Best Mean F1: 0.351
#Best Config: {'anova__k': 11, 'anova__score_func': <function f_classif at 0x000001CD99A33560>, 'lr__C': 0.1, 'lr__max_iter': 1000, 'lr__solver': 'lbfgs', 'scaler': RobustScaler()}


# e este

# grid = {
#         'scaler' : [StandardScaler(),MinMaxScaler(),RobustScaler()],
#         'anova__score_func': [ f_classif],

#         #'anova__score_func': [ f_classif,mutual_info_classif], # mutual_info_classif lenta e não tem melhores resultados
#         'anova__k': [i for i in range(8, 12)],
#         #'anova__k': [15,100,500,1000,1500],  # Number of features to select
#        #'anova__k': [15],  # Number of features to select
        
#        # 'lr__class_weight': ['balanced',None],
#         'lr__max_iter' : [1000],
#      #   'lr__multi_class': ['multinomial'],
#         'lr__solver': ['lbfgs','newton-cg','liblinear'],
#         #'lr__solver': ['lbfgs', 'newton-cg'],  #liblinear not supported in multinomial
#         #'lr__C':[0.01, 0.1] # smaller values means stronger regularizi
#         'lr__C':[1,10,100,1000]
#     }


#Best Mean F1: 0.354
#Best Config: {'anova__k': 8, 'anova__score_func': <function f_classif at 0x000001CD99A33560>, 'lr__C': 100, 'lr__max_iter': 1000, 'lr__solver': 'newton-cg', 'scaler': MinMaxScaler()}

In [60]:
best_pipeline = search.best_estimator_
y_test_results = best_pipeline.predict(X_test)

In [68]:
# submitModel(best_pipeline.predict(X_test),'test_predictions_anova_lr_f1_0354_c01')
# saveModel(best_pipeline,"lr_anova_local_0354_submission_039047")
# loadedm = loadModel("best_models/lr_anova_local_0354_submission_039047.pkl")
